In [230]:
import os
from zoneinfo import ZoneInfo
from datetime import datetime
import h5py

In [231]:
def calc_average(value1, value2):
    return (value1 + value2) / 2

In [232]:
def traverse_group(group, path=''):
    data = {}
    for key, item in group.items():
        new_key = key.lower()
        if isinstance(item, h5py.Dataset):  # Prüfen, ob es sich um einen Datensatz handelt
            data[new_key] = item[()].tolist()  # Konvertieren in eine Liste für MongoDB
            
            #handle inconsistencies (for now only binary data)
            for index, i in enumerate(data[new_key]):
                # decode binary data
                if isinstance(i, (bytes, bytearray)):
                    try:
                        # try parse as float
                        data[new_key][index] = float(i)
                    except ValueError:
                        if i == b'Easteregg :)':
                            print(f"[{new_key}] hit {i}, taking average between last and next value")
                            prev = data[new_key][index-1]
                            next = data[new_key][index+1]
                            # try calculating average between last and next val (does not work on strings)
                            try:
                                data[new_key][index] = calc_average(float(prev), float(next))
                            except ValueError:
                                print("error on calculating average, leaving as decoded string")
                                # on error decode as string
                                data[new_key][index] = str(i.decode())
                                pass
                        else:
                            if new_key == "timestamp":
                                try:
                                    #timestamp in binary data as date string -> convert to unix timestamp
                                    timezone = ZoneInfo("UTC")
                                    date = datetime.strptime(str(i.decode()), '%Y-%m-%dT%H:%M:%S').replace(tzinfo=timezone) #2014-03-24T03:58:1
                                    # print(f"converted to timestamp: {i} is {date.timestamp()}")
                                    data[new_key][index] = date.timestamp()
                                except ValueError:
                                    print(f"error converting to timestamp {i}")
                            else:
                                print(f"[{new_key}] missed binary data {i}, taking average between last and next value")
                                prev = data[new_key][index-1]
                                next = data[new_key][index+1]
                                # try calculating average between last and next val (does not work on strings)
                                try:
                                    data[new_key][index] = calc_average(float(prev), float(next))
                                except ValueError:
                                    print("error on calculating average, leaving as decoded string")
                                    # on error decode as string
                                    data[new_key][index] = str(i.decode())
                                    pass   
                else:
                    # not binary handling
                    #everything that is not float
                    # case does not exist in dataset
                    if not isinstance(i, float):
                        print(f"value is not binary and not float: {i}")
                        
                        
        elif isinstance(item, h5py.Group):  # Prüfen, ob es sich um eine Gruppe handelt
            data[new_key] = traverse_group(item, path + '/' + new_key)  # Rekursiver Aufruf
    return data

In [233]:
def process_h5_file(file_path):
    with h5py.File(file_path, 'r') as h5file:
        data = traverse_group(h5file)  # Start der Rekursion von der Wurzelgruppe

In [234]:
for filename in os.listdir("./dataset"):
    if filename.endswith('.h5'):
        process_h5_file(os.path.join("./dataset", filename))
        

[magnetization] hit b'Easteregg :)', taking average between last and next value
[magnetization] hit b'Easteregg :)', taking average between last and next value
[magnetization] hit b'Easteregg :)', taking average between last and next value
[magnetization] hit b'Easteregg :)', taking average between last and next value
[magnetization] hit b'Easteregg :)', taking average between last and next value
[magnetization] hit b'Easteregg :)', taking average between last and next value
[magnetization] hit b'Easteregg :)', taking average between last and next value
[magnetization] hit b'Easteregg :)', taking average between last and next value
[magnetization] hit b'Easteregg :)', taking average between last and next value
[magnetization] hit b'Easteregg :)', taking average between last and next value
[magnetization] hit b'Easteregg :)', taking average between last and next value
[magnetization] hit b'Easteregg :)', taking average between last and next value
[magnetization] hit b'Easteregg :)', tak

""
